In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR


In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Load datasets
train_dataset = datasets.ImageFolder(root='C:\\ML\\PlantAI\\processed_dataset\\train', transform=transform)
valid_dataset = datasets.ImageFolder(root='C:\\ML\\PlantAI\\processed_dataset\\valid', transform=transform)


In [5]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [6]:
# Define the model (example: ResNet18 pre-trained)
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))  # Adjust final layer for number of classes
model = model.to(device)

In [13]:
import json

# Assuming train_dataset is already defined
class_names = train_dataset.classes
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)


In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)


In [10]:
# Training function
def train_model(model, train_loader, valid_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train, total_train = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = 100 * correct_train / total_train
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%")

        # Validation phase
        model.eval()
        correct_valid, total_valid = 0, 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total_valid += labels.size(0)
                correct_valid += (predicted == labels).sum().item()

        valid_accuracy = 100 * correct_valid / total_valid
        print(f'Validation Accuracy: {valid_accuracy:.2f}%')

        scheduler.step()
        print()

In [12]:
# Train the model
train_model(model, train_loader, valid_loader, epochs=5)


Epoch [1/5], Loss: 0.6791, Training Accuracy: 80.02%
Validation Accuracy: 84.26%

Epoch [2/5], Loss: 0.2754, Training Accuracy: 91.05%
Validation Accuracy: 88.69%

Epoch [3/5], Loss: 0.1794, Training Accuracy: 94.20%
Validation Accuracy: 85.82%

Epoch [4/5], Loss: 0.1619, Training Accuracy: 94.58%
Validation Accuracy: 87.94%

Epoch [5/5], Loss: 0.1148, Training Accuracy: 96.41%
Validation Accuracy: 90.37%



In [13]:
# Save the model
torch.save(model.state_dict(), "plant_disease_model.pth")